In [1]:
from functools import partial

import jax.numpy as jnp
import jax
from jax import lax
from jax.experimental import pallas as pl

array_operand = jnp.array([0.])
lax.cond(True, lambda x: x+1, lambda x: x-1, array_operand)
iter_operand = iter(range(10))
# lax.cond(True, lambda x: x+1, lambda x: next(x)-1, array_operand)

In [8]:
@jax.jit
def test(x):
    return x + 5

In [14]:
jax.make_jaxpr(test)(jnp.zeros(10))

{ lambda ; a:f32[10]. let
    b:f32[10] = pjit[
      name=test
      jaxpr={ lambda ; c:f32[10]. let d:f32[10] = add c 5.0 in (d,) }
    ] a
  in (b,) }

In [3]:
def matmul_kernel(x_ref, y_ref, o_ref, *, activation, block_k):
  acc = jnp.zeros((x_ref.shape[0], y_ref.shape[1]), jnp.float32)
  for k in range(x_ref.shape[1] // block_k):
    x = x_ref[:, k*block_k:(k+1)*block_k]
    y = y_ref[k*block_k:(k+1)*block_k, :]
    acc += x @ y
  o_ref[:, :] = activation(acc).astype(o_ref.dtype)

x, y = jnp.ones((512, 256)), jnp.ones((256, 1024))
block_shape = 64, 256, 64

@partial(jax.jit, static_argnames=["block_shape", "activation"])
def matmul(x, y, *, block_shape, activation):
  block_m, block_n, block_k = block_shape
  fused_matmul = pl.pallas_call(
      partial(matmul_kernel, block_k=block_k, activation=activation),
      out_shape=jax.ShapeDtypeStruct((x.shape[0], y.shape[1],), jnp.float32),
      in_specs=[
        pl.BlockSpec(lambda i, j: (i, 0), (block_m, x.shape[1])),
        pl.BlockSpec(lambda i, j: (0, j), (y.shape[0], block_n))
      ],
      out_specs=pl.BlockSpec(lambda i, j: (i, j), (block_m, block_n)),
      grid=(4, 4),
  )
  return fused_matmul(x, y)


In [8]:
matmul(x, y, block_shape=block_shape, activation=jax.nn.gelu)
%timeit matmul(x, y, block_shape=block_shape, activation=jax.nn.gelu)

84.9 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
@partial(jax.jit, static_argnums=(2,))
def fused_matmul(x, y, activation):
    return activation(jnp.matmul(x, y))

fused_matmul(x, y, jax.nn.gelu)
%timeit fused_matmul(x, y, jax.nn.gelu)

61 µs ± 7.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
